In [1]:
from LeptonUtils import *
import time
import cv2

In [2]:
def frame_diff(new_img, running_avg, alpha=0.1):
    # Difference
    out = cv2.absdiff(new_img, running_avg)

    # Add to average
    running_avg = cv2.addWeighted(running_avg, 1-alpha, new_img, alpha, 0)

    return out, running_avg

In [3]:
# backSub = cv2.createBackgroundSubtractorMOG2(varThreshold=80, detectShadows=False)
backSub = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=800, detectShadows=False)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,4))

In [11]:
cv2.namedWindow("bg", cv2.WINDOW_NORMAL)
cv2.namedWindow("delta", cv2.WINDOW_NORMAL)

vid = Raw16Video("vids/Lepton_Capture_5.tiff")
ret, first_frame = vid.read()
print(first_frame.dtype)

# first_frame = clip_norm(first_frame, temp2raw(22), temp2raw(24))

# Create video writer object to save processed video
video_writer = cv2.VideoWriter(
    filename=f"./out/rec_{int(time.time())}.mp4",
    apiPreference=cv2.CAP_FFMPEG,
    fourcc=cv2.VideoWriter_fourcc(*'XVID'),
    fps=8.0,
    frameSize=(160, 120),
    params=[
        cv2.VIDEOWRITER_PROP_IS_COLOR, 0
    ],
)
assert video_writer.isOpened()

cv2.undistort()

while cv2.waitKey(1) & 0xFF != ord('q'):
    ret, frame = vid.read()
    if not ret: break
    cv2.imshow("bg", clip_norm(frame, temp2raw(25.5), temp2raw(27)))

    # subtracted = backSub.apply(frame)
    frame = cv2.medianBlur(frame, 3)
    # frame = clip_norm(frame, temp2raw(22), temp2raw(24))
    frame, first_frame = frame_diff(frame, first_frame, 0.2)

    frame = clip_norm(frame, min_val=20)
    
    # frame = cv2.threshold(frame, 5, 2**16, cv2.THRESH_BINARY)[1]
    # frame = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)

    time.sleep(1/10)
    cv2.imshow("delta", frame)
    video_writer.write(frame)

video_writer.release()
cv2.destroyAllWindows()

uint16
